# Loading meetup data

In [ ]:
import os
import json
from typing import Tuple
from requests import Response
from functools import partial
from ipywidgets import widgets, interact_manual, ToggleButtons
from IPython.display import display, Code, Markdown, HTML, update_display, TextDisplayObject, display_html, Pretty

# project imports
from meetup.api import Client, RequestException

## Meetup api key

Requests to the meetup.com api need to be authenticated. The easiest way is to grab an api key that allows you to use the endpoints with your meetup account.

Get your meetup api key from https://secure.meetup.com/meetup_api/key/ and set it as `MEETUP_API_KEY`.

In [ ]:
MEETUP_API_KEY = os.environ["MEETUP_API_KEY"]

# Create a Client instance with the api key
client = Client(MEETUP_API_KEY)

# make a test call to the '/dashboard' endpoint.
response = client.get("/dashboard")
fmt = response.status_code == 200 and "**" or "~~"

# render a success message in markdown
display(Markdown(f"{fmt}IT WORKED{fmt}"))

# render the response text as json code
display(Code(response.text, language="json"))

### Create a MeetupGroup



The simplest way to create input elements is to use `ipywidgets.widgets.interact_manual` as decorator.
It will render input fields depending on the used kwargs together with an "Run interact" button that will call the decorated function when pressed.

In [ ]:
def create_group(data: dict) -> Tuple[MeetupGroup, bool]:
    """
    Create or update a group with the data from the group enpoint.
    """
    return MeetupGroup.objects.update_or_create(
        urlname=data["urlname"], 
        defaults={
            "uid":data["id"],
            "name":data["name"],
            "link":data["link"],
            "members_count":data["members"]
        }
    )


def create_event(data: dict, group: MeetupGroup) -> Tuple[MeetupEvent, bool]:
    """
    Create or update a event with the data from the group enpoint.
    """
    return MeetupEvent.objects.update_or_create(
        uid=data["id"],
        defaults={
            "group": group,
            "name": data["name"],
            "description": data.get("description",""),
            "event_date": data["local_date"],
            "rsvp_count": data["yes_rsvp_count"],
            "link": data["link"],
            "venue": data["venue"]["name"],
        }
    )


def create_member(data: dict) -> Tuple[MeetupMember, bool]:
    """
    Create or update a Meetup Member.
    """
    return MeetupMember.objects.update_or_create(
        uid=data["id"],
        defaults={
            "name": data["name"],
            "photo": data.get("photo", {}).get("thumb_link", None)
        }
    )


In [ ]:
@interact_manual(urlname="Django-Vienna")
def view_group(urlname: str) -> MeetupGroup:

    try:
        data = client.get_group(urlname)
    except RequestException as e:
        display(Code(e.args[0], language="json"))
        return

    displays = {
        "Description": HTML(data["description"]),
        "Json": Code(json.dumps(data, indent=4), language="json")
    }
    
    toggles = ToggleButtons(options=displays.keys(), value=list(displays.keys())[0])
    display(Markdown(f"## [{data['name']}]({data['link']})"))
    display(toggles)
    display(None, display_id="response")
    
    def udpate_response(event):
        update_display(displays[toggles.value], display_id="response")

    toggles.observe(udpate_response)
    udpate_response(None)

    exists = MeetupGroup.objects.filter(urlname=urlname).exists()
    button_save = widgets.Button(description=f"{exists and 'update' or 'create'} group")
    display(button_save)

    def _create(btn):
        group, created = create_group(data)
        display(f"Meetupgroup {group} {created and 'created' or 'updated'}.")

    button_save.on_click(_create)


We should end up with a MeetupGroup instance.

In [ ]:
MeetupGroup.objects.all()

### Create the MeetupEvents and their MeetupMembers of the meetup group

In [ ]:
def get_group_events(group):
    member_cache = {}

    for event_data in client.get_group_events(group.urlname):
        # create the event object
        event, created = create_event(event_data, group)
        action = created and "created" or "updated"
        display(Pretty(f"{action} MeetupEvent {event.name}"))
        
        # load the attendends
        for attendance in client.get_event_attendance(group.urlname, event.uid):
            if attendance["rsvp"]["response"] != "yes":
                print(f'skipping {attendance["member"]["name"]}: rsvp was "{attendance["rsvp"]["response"]}"')
                continue

            member_data = attendance["member"]
            if member_data["id"] in member_cache:
                member = member_cache[member_data["id"]]
            else:
                member, created = create_member(member_data)
                member_cache[member_data["id"]] = member

            event.attendants.add(member)
            print(f'added {member.name}.')
                

In [ ]:
get_group_events(MeetupGroup.objects.first())

In [ ]:
MeetupEvent.objects.all()

In [ ]:
MeetupMember.objects.all()

In [ ]:
MeetupMember.objects.count()